In [54]:
# -*- coding: utf-8 -*-
"""
Created on Wed Feb  7 11:24:13 2024

@author: mrsag
"""

import yt
import numpy as np
import matplotlib.pyplot as plt
import glob
from Curve_fitting_with_scipy import Gaussianfitting as Gf
from scipy.signal import fftconvolve
from matplotlib.offsetbox import OffsetImage, AnnotationBbox
import matplotlib.image as mpimg
from matplotlib.legend_handler import HandlerBase
from matplotlib.lines import Line2D
from matplotlib.path import Path
from skimage import io
from skimage.transform import resize
from mpl_toolkits.mplot3d import Axes3D
import plotly.graph_objects as go

import matplotlib as mpl


mpl.rcParams['font.family'] = 'serif'
mpl.rcParams['font.serif'] = 'Times New Roman'
mpl.rcParams['font.size'] = 12
#mpl.rcParams['font.weight'] = 'bold'
#mpl.rcParams['font.style'] = 'italic'  # Set this to 'italic'
mpl.rcParams['figure.dpi']=100 # highres display

In [55]:

def find_index(array, value):
    # Calculate the absolute differences between each element and the target value
    absolute_diff = np.abs(array - value)
    
    # Find the index of the minimum absolute difference
    index = np.argmin(absolute_diff)
    
    return index

def moving_average(signal, window_size):
    # Define the window coefficients for the moving average
    window = np.ones(window_size) / float(window_size)
    
    # Apply the moving average filter using fftconvolve
    filtered_signal = fftconvolve(signal, window, mode='same')
    
    return filtered_signal


In [56]:
class SimulationBox3D:
    def __init__(self, resolution_x=50, resolution_y=50, resolution_z=50, potential_offset=0):
        self.resolution_x = resolution_x
        self.resolution_y = resolution_y
        self.resolution_z = resolution_z

        self.x, self.y, self.z = np.mgrid[0:1:resolution_x*1j,
                                          0:1:resolution_y*1j,
                                          0:1:resolution_z*1j]
        self.potential = np.full_like(self.x, potential_offset, dtype=float)
        self.fixed_mask = np.zeros_like(self.potential, dtype=bool)

    def add_sphere(self, center, radius, potential=0):
        cx, cy, cz = center
        mask = ((self.x - cx)**2 + (self.y - cy)**2 + (self.z - cz)**2) <= radius**2
        self.potential[mask] = potential
        self.fixed_mask[mask] = True

    def add_box(self, x_bounds, y_bounds, z_bounds, potential=0):
        x0, x1 = x_bounds
        y0, y1 = y_bounds
        z0, z1 = z_bounds
        mask = ((self.x >= x0) & (self.x <= x1) &
                (self.y >= y0) & (self.y <= y1) &
                (self.z >= z0) & (self.z <= z1))
        self.potential[mask] = potential
        self.fixed_mask[mask] = True

    def add_cylinder(self, base_center, radius, height, axis='z', potential=0):
        cx, cy = base_center
        if axis == 'z':
            mask = (((self.x - cx)**2 + (self.y - cy)**2 <= radius**2) & (self.z <= height))
        elif axis == 'x':
            mask = (((self.y - cy)**2 + (self.z - cx)**2 <= radius**2) & (self.x <= height))
        elif axis == 'y':
            mask = (((self.x - cx)**2 + (self.z - cy)**2 <= radius**2) & (self.y <= height))
        else:
            raise ValueError("axis must be 'x', 'y', or 'z'")
        self.potential[mask] = potential
        self.fixed_mask[mask] = True

    def add_ellipsoid(self, center, radii, potential=0):
        cx, cy, cz = center
        rx, ry, rz = radii
        mask = (((self.x - cx)/rx)**2 + ((self.y - cy)/ry)**2 + ((self.z - cz)/rz)**2 <= 1)
        self.potential[mask] = potential
        self.fixed_mask[mask] = True

    def add_hyperboloid(self, center, coeffs, threshold=1, potential=0):
        cx, cy, cz = center
        a, b, c = coeffs
        mask = (((self.x - cx)/a)**2 + ((self.y - cy)/b)**2 - ((self.z - cz)/c)**2 <= threshold)
        self.potential[mask] = potential
        self.fixed_mask[mask] = True

    def solve(self, max_iter=1000, tol=1e-4, method='jacobi', verbose=False):
        V = self.potential.copy()
        for it in range(max_iter):
            V_old = V.copy()
            if method == 'jacobi':
                V_new = V.copy()
                V_new[1:-1, 1:-1, 1:-1] = 1/6 * (
                    V[2:, 1:-1, 1:-1] + V[:-2, 1:-1, 1:-1] +
                    V[1:-1, 2:, 1:-1] + V[1:-1, :-2, 1:-1] +
                    V[1:-1, 1:-1, 2:] + V[1:-1, 1:-1, :-2]
                )
                mask = ~self.fixed_mask[1:-1, 1:-1, 1:-1]
                V[1:-1, 1:-1, 1:-1][mask] = V_new[1:-1, 1:-1, 1:-1][mask]
            elif method == 'gauss-seidel':
                for i in range(1, self.resolution_x - 1):
                    for j in range(1, self.resolution_y - 1):
                        for k in range(1, self.resolution_z - 1):
                            if not self.fixed_mask[i, j, k]:
                                V[i, j, k] = 1/6 * (V[i+1, j, k] + V[i-1, j, k] +
                                                   V[i, j+1, k] + V[i, j-1, k] +
                                                   V[i, j, k+1] + V[i, j, k-1])
            else:
                raise ValueError("Unknown method")

            diff = np.max(np.abs(V - V_old))
            if verbose and it % 50 == 0:
                print(f"Iteration {it}, max change: {diff:.2e}")
            if diff < tol:
                if verbose:
                    print(f"Converged at iteration {it}, max change: {diff:.2e}")
                break
        self.potential = V



In [57]:
# Create simulation box and add all geometries
box = SimulationBox3D(100, 100, 100)

# box.add_sphere(center=(0.25, 0.25, 0.25), radius=0.1, potential=1)
# box.add_box((0.6, 0.8), (0.6, 0.8), (0.6, 0.8), potential=-1)
box.add_cylinder(base_center=(0.5, 0.5), radius=0.1, height=0.9, axis='z', potential=0.5)
# box.add_ellipsoid(center=(0.75, 0.25, 0.25), radii=(0.05, 0.1, 0.1), potential=0.8)
box.add_hyperboloid(center=(0.1, 0.1, 0.5), coeffs=(0.2, 0.2, 0.4), threshold=1, potential=0.8)
box.add_hyperboloid(center=(0.9, 0.1, 0.5), coeffs=(0.1, 0.2, 0.4), threshold=1, potential=-0.8)
box.add_hyperboloid(center=(0.1, 0.9, 0.5), coeffs=(0.2, 0.1, 0.4), threshold=1, potential=-0.8)
box.add_hyperboloid(center=(0.9, 0.9, 0.5), coeffs=(0.1, 0.1, 0.4), threshold=1, potential=0.8)

# Solve potential field
box.solve(max_iter=300, tol=1e-4, method='gauss-seidel', verbose=True)




Iteration 0, max change: 4.93e-01
Iteration 50, max change: 3.82e-03
Iteration 100, max change: 2.23e-03
Iteration 150, max change: 1.59e-03
Iteration 200, max change: 1.21e-03
Iteration 250, max change: 9.62e-04


In [58]:
# Visualize potential in 3D (central slice)
# z_slice = 25
# fig = plt.figure(figsize=(8, 6))
# ax = fig.add_subplot(111, projection='3d')
# X, Y = np.meshgrid(np.linspace(0, 1, box.resolution_x),
#                    np.linspace(0, 1, box.resolution_y))
# Z = box.potential[:, :, z_slice]
# ax.plot_surface(X, Y, Z, cmap='viridis')
# ax.set_title('3D Potential Field (Z-Slice)')
# plt.tight_layout()
# plt.show()

In [59]:
# Normalize potential values for better visualization (optional)
potential = box.potential


# Define custom colorscale mimicking "jet" colormap
colors = [
    [0, "rgb(0,0,128)"],   # Dark blue
    [0.25, "rgb(0,0,255)"], # Blue
    [0.5, "rgb(0,255,255)"], # Cyan
    [0.75, "rgb(255,255,0)"], # Yellow
    [1, "rgb(255,0,0)"]   # Red
]
# Create volume plot
fig = go.Figure(data=go.Volume(
    x=np.repeat(np.linspace(0, 1, box.resolution_x), box.resolution_y * box.resolution_z),
    y=np.tile(np.repeat(np.linspace(0, 1, box.resolution_y), box.resolution_z), box.resolution_x),
    z=np.tile(np.linspace(0, 1, box.resolution_z), box.resolution_x * box.resolution_y),
    value=potential.flatten(),
    opacity=0.1,  # Lower values make the plot more transparent
    surface_count=20,  # Number of isosurfaces
    colorscale=colors
))

fig.update_layout(
    scene=dict(
        xaxis_title='X',
        yaxis_title='Y',
        zaxis_title='Z'
    ),
    title='3D Potential Field Density Plot',
    margin=dict(l=0, r=0, b=0, t=30)
)

fig.show(renderer="browser")